## CMIP6 sea ice thickness bias

## Import libraries

In [ ]:
import warnings

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download, plot

plt.style.use("seaborn-v0_8-notebook")
warnings.filterwarnings("ignore", module="cf_xarray")

## Set parameters

In [ ]:
year_start = 2002
year_stop = 2014
assert year_start >= 2002 and year_stop <= 2014

# Choose CMIP6 historical models
models = [
    "access_cm2",
    "access_esm1_5",
    "canesm5",
    "cmcc_cm2_sr5",
    "cmcc_esm2",
    "cnrm_cm6_1",
    "cnrm_cm6_1_hr",
    "cnrm_esm2_1",
    "e3sm_1_0",
    "e3sm_1_1",
    "e3sm_1_1_eca",
    "ec_earth3_aerchem",
    "ec_earth3_cc",
    "ec_earth3_veg_lr",
    "hadgem3_gc31_ll",
    "ipsl_cm5a2_inca",
    "ipsl_cm6a_lr",
    "miroc6",
    "miroc_es2l",
    "mpi_esm1_2_hr",
    "mpi_esm1_2_lr",
    "nesm3",
    "norcpm1",
    # "taiesm1",  # very large values
    "ukesm1_0_ll",
]

## Define request

In [ ]:
months = [f"{month:02d}" for month in [1, 2, 3, 4, 10, 11, 12]]
year_ranges = {
    "envisat": range(max(2002, year_start), min(2010, year_stop) + 1),
    "cryosat_2": range(max(2010, year_start), min(2020, year_stop) + 1),
}
collection_id_satellite = "satellite-sea-ice-thickness"
request_satellite = {
    "version": "2_0",
    "cdr_type": "cdr",
    "variable": "all",
    "month": months,
}

collection_id_cmip6 = "projections-cmip6"
request_cmip6 = {
    "format": "zip",
    "temporal_resolution": "monthly",
    "experiment": "historical",
    "variable": "sea_ice_thickness",
    "month": months,
}

chunks = {"year": 1}

## Define function to cache

In [ ]:
def regridded_monthly_weighted_mean(ds, times, **kwargs):
    ds_sat = download.download_and_transform(
        "satellite-sea-ice-thickness",
        {
            "satellite": "envisat",
            "version": "2_0",
            "cdr_type": "cdr",
            "variable": "all",
            "year": "2002",
            "month": [f"{month:02d}" for month in [1, 2, 3, 4, 10, 11, 12]],
        },
        chunks={"year": 1},
    )
    ds["time"] = pd.to_datetime(ds["time"].dt.strftime("%Y-%m"))
    ds["time"].attrs["standard_name"] = "time"
    ds = ds.sel(time=times)
    ds = diagnostics.monthly_weighted_mean(ds)
    return diagnostics.regrid(ds, ds_sat[["latitude", "longitude"]], **kwargs)

## Download and transform

In [ ]:
datasets_satellite = []
datasets_cmip6 = []
for satellite, year_range in year_ranges.items():
    print(f"{satellite=}")
    years = list(map(str, year_range))
    ds = download.download_and_transform(
        collection_id_satellite,
        request_satellite | {"satellite": satellite, "year": years},
        chunks=chunks,
    )
    times = ds["time"].dt.strftime("%Y-%m").values.tolist()
    ds = diagnostics.monthly_weighted_mean(ds)
    datasets_satellite.append(
        ds.expand_dims(satellite=[satellite], product=["satellite"])
    )

    model_datasets = []
    for model in models:
        print(f"{satellite=} {model=}")
        ds = download.download_and_transform(
            collection_id_cmip6,
            request_cmip6 | {"model": model, "year": years},
            chunks=chunks,
            transform_chunks=False,
            transform_func=regridded_monthly_weighted_mean,
            transform_func_kwargs={
                "times": times,
                "method": "nearest_s2d",
                "periodic": True,
                "ignore_degenerate": True,
            },
        )
        model_datasets.append(
            ds.expand_dims(model=[model], satellite=[satellite], product=["cmip6"])
        )
    datasets_cmip6.append(
        xr.concat(model_datasets, "model").mean("model", keep_attrs=True)
    )
da_satellite = xr.concat(datasets_satellite, "satellite")["sea_ice_thickness"].rename(
    "sithick"
)
da_cmip6 = xr.concat(datasets_cmip6, "satellite")["sithick"]

## Plot monthly maps

In [ ]:
projection = ccrs.Stereographic(central_latitude=90)
da = xr.concat([da_satellite, da_cmip6], "product")
da = da.where(da.sel(product="satellite").notnull())
for dim in ds.dims:
    da = da.dropna(dim, how="all")
for month, da_month in da.groupby("month"):
    facet = plot.projected_map(
        da_month,
        col="product",
        row="satellite",
        vmin=0,
        robust=True,
        projection=projection,
    )
    facet.fig.suptitle(f"{month=}", y=1.01)
    plt.show()

## Plot bias

In [ ]:
with xr.set_options(keep_attrs=True):
    bias = da.sel(product="cmip6") - da.sel(product="satellite")
plot.projected_map(
    bias, col="satellite", row="month", projection=projection, robust=True
)